In [19]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader 
from transformers import DistilBertTokenizer, DistilBertModel
import torch

In [18]:
from torch import cuda
device = torch.device('cuda' if cuda.is_available() else 'cpu')

print(f"Current device: {device}")

Current device: cpu


In [22]:
train_path = 'data/input/train.csv'
test_path = 'data/input/test.csv'


label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

print(f"Total Training Records : {len(df_train)}")

df_train.drop(['id'], inplace=True, axis=1)
df_train['labels'] = df_train.iloc[:, 1:].values.tolist()
df_train.drop(df_train.columns.values[1:-1].tolist(), inplace=True, axis=1)

df_train["comment_text"] = df_train["comment_text"].str.lower()
df_train["comment_text"] = df_train["comment_text"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")

df_train.head()

Total Training Records : 159571


,comment_text,labels
0,explanation why the edits made under my userna...,"[0, 0, 0, 0, 0, 0]"
1,d'aww! he matches this background colour i'm s...,"[0, 0, 0, 0, 0, 0]"
2,"hey man, i'm really not trying to edit war. it...","[0, 0, 0, 0, 0, 0]"
3,""" more i can't make any real suggestions on im...","[0, 0, 0, 0, 0, 0]"
4,"you, sir, are my hero. any chance you remember...","[0, 0, 0, 0, 0, 0]"


In [15]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
EPOCHS = 8
LEARNING_RATE = 1e-05
NUM_WORKERS = 2